In [55]:
from pearce.emulator.emu import OriginalRecipe, ExtraCrispy
from pearce.emulator.trainingData import parameter, PARAMS

In [56]:
from george.kernels import ExpSquaredKernel
import george

In [57]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from itertools import cycle
colors = cycle(sns.color_palette())

In [58]:
training_dir = '/u/ki/swmclau2/des/PearceLHC/'

In [59]:
or_params = PARAMS[:]
or_params.append(parameter('r', 0, 1)) #95% sure bounds aren't used for r
emu = OriginalRecipe(training_dir,or_params)
#emu = ExtraCrispy(training_dir)

In [60]:
emu.yerr*=100
emu.build_emulator('xi', {})

In [61]:
ig = emu.get_initial_guess('xi', {})
a = ig['amp']
kernel = a * ExpSquaredKernel(emu.metric, ndim=emu.emulator_ndim)
loo_gp = george.GP(kernel)
# gp = george.GP(kernel, solver=george.HODLRSolver, nleaf=x.shape[0]+1,tol=1e-18)

loo_gp.compute(emu.x[1:], emu.yerr[1:])  # NOTE I'm using a modified version of george!

In [62]:
K = emu.gp.kernel.value(emu.x)
print np.diag(K)
print emu.yerr

[ 0.481  0.481  0.481 ...,  0.481  0.481  0.481]
[ 1.03807372  0.31082861  0.23575004 ...,  0.42593738  1.0945215
  3.60801311]


In [63]:
K_inv_full = emu.gp.solver.apply_inverse(np.eye(emu.x.shape[0]),in_place=True)

In [64]:
y_param = 'logMmin'
emulation_point = [('f_c', 0.233), ('logM0', 12.0), ('sigma_logM', 0.533), 
                   ('alpha', 1.083),('logM1', 13.5), ('logMmin', 12.233)]
em_params = dict(emulation_point)
#del em_params[y_param]

In [65]:
from os import path
save_dir = '/u/ki/swmclau2/des/EmulatorMCMC/'
rpoints = np.log10(np.loadtxt(path.join(save_dir, 'rpoints.npy')) )
true_vals = np.log10( np.loadtxt(path.join(save_dir, 'xi.npy')) )

In [66]:
rbins = np.array([  0.06309573,   0.12437607,   0.24517359,   0.34422476, 0.48329302, 0.67854546,\
           0.9526807 , 1.33756775,1.8779508 ,   2.6366509 ,   3.70186906,   5.19743987, 7.29722764, \
           10.24533859,  14.38449888,  20.1958975 , 28.35512583,  39.81071706] )
#rpoints= np.log10((rbins[1:]+rbins[:-1])/2)
em_params['r'] = rpoints#[:2]

In [67]:
input_params = {}
input_params.update(em_params)

# i'd like to remove 'r'. possibly requiring a passed in param?
t_list = [input_params[p.name] for p in emu.ordered_params]
t_grid = np.meshgrid(*t_list)
t = np.stack(t_grid).T
t = t.reshape((-1, emu.emulator_ndim))
try:
    t = np.sort(t.view(','.join(['float64' for _ in xrange(min(t.shape))])),
                order=['f%d' % i for i in xrange(min(t.shape))], axis=0).view(np.float)
except ValueError:
        t = np.sort(t.view(','.join(['float64' for _ in xrange(max(t.shape))])),
                order=['f%d' % i for i in xrange(max(t.shape))], axis=0).view(np.float)

/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:10: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead


In [68]:
print t.shape

(17, 7)


In [69]:
print emu.y.shape

(8432,)


In [70]:
#loo_mu = loo_gp.predict(emu.y[1:, 0], t , mean_only=True)
loo_mu = loo_gp.predict(emu.y[1:], t , mean_only=True)
print loo_mu

[ 3.27478415  2.91670686  2.53932793  2.1973586   1.87049094  1.49046265
  1.09514301  0.75238009  0.48984533  0.26512903  0.022259   -0.20933873
 -0.474599   -0.75575697 -1.06897751 -1.39684355 -1.90324827]


In [71]:
loo_inv = loo_gp.solver.apply_inverse(np.eye(emu.x.shape[0]-1),in_place=True)
print loo_inv.shape, K_inv_full.shape

(8431, 8431) (8432, 8432)


In [72]:
print loo_inv[0,:]
print 
print loo_inv[1,:]

[  8.21877378e+00  -1.16311768e+00  -1.28117866e+00 ...,  -3.97088371e-07
  -2.12451558e-07  -1.01048544e-06]

[ -1.16311768e+00   1.26769228e+01  -6.00170946e-01 ...,   3.69837241e-07
   4.92379426e-07   3.01231946e-06]


In [73]:
print K_inv_full[0,:]
print 
print K_inv_full[1,:]

[  8.41638316e-01  -9.33577726e-02   2.88365335e-02 ...,  -1.94920108e-07
   3.80550986e-08   1.70124510e-08]

[ -9.33577726e-02   7.16358940e+00  -1.43786314e+00 ...,   3.22850158e-06
  -2.06993635e-07  -9.54387521e-08]


In [74]:
mu, G = emu.emulate(em_params)
print mu

[ 3.506215    2.94238421  2.55057474  2.23116454  1.89619466  1.53034952
  1.1433539   0.77875163  0.49230745  0.26154951  0.02023064 -0.20623929
 -0.47746181 -0.73868938 -1.02035267 -1.50507731 -1.79546714]


In [75]:
yhat = loo_gp.predict(emu.y[1:], emu.x[0].reshape((1,emu.x.shape[1])), mean_only=True)
#yhat = loo_gp.predict(emu.y[1:, 0], emu.x[0].reshape((1,emu.x.shape[1])), mean_only=True)

print yhat

[ 3.65551011]


In [76]:
#print emu.y[0, 0]
print emu.y[0]

3.72177668084


In [77]:
x, y = emu.x[:], emu.y[:]

N = K_inv_full.shape[0]

In [78]:
from time import time
from scipy.linalg import inv

In [79]:
my_inds = [i for i in xrange(1, N-1)]
my_inds.append(0)
my_inds = np.array(my_inds)

In [80]:
from copy import deepcopy

In [81]:
times = np.zeros((N,))

In [82]:
mus = np.zeros((N, len(rpoints)))

In [ ]:
t0 = time()
for idx in xrange(N):
    print idx
    #TODO put switches at the end of the loop too
    #then i can make just one deepcopy
    #t_x = deepcopy(emu.x)
    t_x = emu.x
    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    
    #t_yerr = deepcopy(emu.yerr)
    t_yerr = emu.yerr
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    
    #t_y = deepcopy(emu.y[:, 0])
    t_y = emu.y
    #t_y[[N-1, idx], :] = t_y[[idx, N-1], :]
    t_y[[N-1, idx]] = t_y[[idx, N-1]]
    #K_inv = deepcopy(K_inv_full)
    K_inv = K_inv_full
    
    K_inv[[idx, N - 1], :] = K_inv[[N - 1, idx], :]
    K_inv[:, [idx, N - 1]] = K_inv[:, [N - 1, idx]]
    
    K_m_idx_inv = K_inv[:N - 1, :][:,:N - 1] \
            -np.outer(K_inv[N-1,:N-1], K_inv[:N-1,N-1])/K_inv[N-1,N-1]
           
    #alpha_m_idx = [np.dot(K_m_idx_inv, t_y[:N-1, rbin ]-emu.gp.mean(t_x[:N-1])) for rbin in xrange(t_y.shape[1]) ]
    alpha_m_idx = np.dot(K_m_idx_inv, t_y[:N-1 ]-emu.gp.mean(t_x[:N-1]))

    Kxxs_t = emu.gp.kernel.value(t, t_x[:N-1])
    #mus[idx, :] = np.array([np.dot(Kxxs_t, alpha_m_idx[rbin])+ emu.gp.mean(t) for rbin in xrange(t_y.shape[1]) ])[:,0]
    mus[idx, :] = np.dot(Kxxs_t, alpha_m_idx)+ emu.gp.mean(t)

    #print mus[idx]
    #print
    K_inv_full[[idx, N - 1], :] = K_inv_full[[N - 1, idx], :]
    K_inv_full[:, [idx, N - 1]] = K_inv_full[:, [N - 1, idx]]

    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    #t_y[[N-1, idx],:] = t_y[[idx, N-1],:]
    t_y[[N-1, idx]] = t_y[[idx, N-1]]
print time()-t0, 's'

0
1

In [ ]:
np.diag((N-1.0)/N*np.cov(mus, rowvar=False))

In [ ]:
t0 = time()
for idx in xrange(N):
    print idx
    #TODO put switches at the end of the loop too
    #then i can make just one deepcopy
    #t_x = deepcopy(emu.x)
    t_x = emu.x
    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    
    #t_yerr = deepcopy(emu.yerr)
    t_yerr = emu.yerr
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    
    #t_y = deepcopy(emu.y[:, 0])
    t_y = emu.y
    #t_y[[N-1, idx], :] = t_y[[idx, N-1], :]
    t_y[[N-1, idx]] = t_y[[idx, N-1]]
    loo_gp.compute(t_x[:N-1], t_yerr[:N-1]) 
    mus[idx,:] = np.array([loo_gp.predict(emu.y[:N-1, rbin], t , mean_only=True) for rbin in xrange(t_y.shape[1]) ])[:,0]

    #print
    #K_inv_full[[idx, N - 1], :] = K_inv_full[[N - 1, idx], :]
    #K_inv_full[:, [idx, N - 1]] = K_inv_full[:, [N - 1, idx]]

    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    #t_y[[N-1, idx],:] = t_y[[idx, N-1],:]
    t_y[[N-1, idx]] = t_y[[idx, N-1]]
print time()-t0, 's'

In [ ]:
np.diag((N-1.0)/N*np.cov(mus, rowvar=False))

In [ ]:
print G